**FIT3183 Malicious AI & Dark Side Security**
# Week 5 Advanced Lab: Deepfakes I

This notebook demonstrates implementation of Deepfakes with First Order Motion Model for Image Animation and Generative Adversarial Networks for Face Swapping.

First, check out the recent Nvidia's Deep Fake of CEO Jensen Huang at GTC [(Behind the Scenes)](https://www.youtube.com/watch?v=f_V30ueEXE4)!

There are 3 task this week:

1. Task 1: **First Order Motion Model for Image Animation** 
- Demo for Animation, FaceSwap, Wav2Lip
2. Task 2: Part Swap Demo for **Motion Supervised co-part Segmentation**
3. (Optional) Advanced Task 3: **DeepFake Lite Training with DeepFaceLab**
4. (Optional) Advanced Task 4: **Motion Representations for Articulated Animation**

Some Deepfake Video Examples:
* [The Mona Lisa and Elon Musk](https://www.youtube.com/watch?v=GkOA0X-A58Q)
* [Kim Kardashian and Dr. K](https://www.youtube.com/watch?v=QmdbtEPlOVA)
* [Woman and Edward Snowden](https://www.youtube.com/watch?v=wSlC39NYenY)

Deepfakes are faked media that mimic another person's likeness. This process is achieved through something called "deep learning," hence the name deepfake.

Deepfakes Behind the Scenes:
1. Extraction
- Face Detection
- Face Alignment
- Face Segmentation
2. Training
3. Conversion
- Blending
- Sharpening

*OpenSource:*

[DeepFaceLab](https://github.com/iperov/DeepFaceLab) - the leading software for creating deepfakes

Paper Reference: https://arxiv.org/abs/2005.05535

👉 *Copy this Colab notebook to your Drive*, read the instruction and fill the missing code.

👉 *Use GPU:* `Runtime > Change runtime type > GPU`.


<small>*Prepared by [Yin Yin Low](mailto:yin.low@monash.edu) (Lab Tutor) Aug 2022.*

## Task 4: Motion Representations for Articulated Animation

Paper reference: https://arxiv.org/abs/2104.11280

**Overview**

The region predictor returns heatmaps for each part in the source and the driving images. We then compute principal axes of each heatmap, to transform each region from the source to the driving frame through a whitened reference frame. Region and background transformations are combined by the pixel-wise flow prediction network. The target image is generated by warping the source image in the feature space using the pixel-wise flow, and inpainting newly introduced regions, as indicated by the confidence map.

<img src="https://snap-research.github.io/articulated-animation/resources/framework.png">

**Example Animation**
Here is an example of several images produced by motion representations for articulated animation method. In the first column the driving video is shown. For the remaining columns the top image is animated by using motions extracted from the driving.
<img src="https://github.com/snap-research/articulated-animation/raw/main/sup-mat/teaser.gif">

In [ ]:
!rm -rf /content/articulated-animation

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install
!git clone https://github.com/snap-research/articulated-animation.git

In [ ]:
cd articulated-animation

### Step 1: Load source image and driving video

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('sup-mat/source.png')
driving_video = imageio.mimread('sup-mat/driving.mp4')

source_image = resize(source_image, (384, 384))[..., :3]
driving_video = [resize(frame, (384, 384))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    
## YOUR CODE STARTS HERE ##
# Display the source image and driving video with HTML function (1 line)

## YOUR CODE ENDS HERE ##

### Step 2: Load checkpoints

In [ ]:
from demo import load_checkpoints

generator, region_predictor, avd_network = load_checkpoints(config_path='config/ted384.yaml',
                                                            checkpoint_path='checkpoints/ted384.pth')

### Step 3: Perform Animation

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, 
                             region_predictor, avd_network, animation_mode='avd')

#save resulting video
imageio.mimsave('generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

## YOUR CODE STARTS HERE ##
# Display the source image, driving video and predictions with HTML function (1 line)

## YOUR CODE ENDS HERE ##